In [1]:
%load_ext autoreload
%autoreload 2

import random
import numpy as np
import pandas as pd
import IPython.display as dp

from jassbot.Config import *
from jassbot.GameLogic import *
from jassbot.GameStrategy import *

create_logger()

Using Theano backend.


In [ ]:
state = init()
new_game(state)
print_state(state)
while len(state['played']) < 36:
    play_round(state)
print_state(state)

In [ ]:
state = init()
new_game(state)
print_state(state)
play_round(state, lambda cards: random.choice(cards))
play_round(state, lambda cards: random.choice(cards))
print_state(state)

In [2]:
state = init()
epsilon = 1
epochs = 1000
model = create_model()
test_runs = 100

simulations = 2
logger = get_logger()

stats = pd.DataFrame({'rounds won':[], 'games won':[]})
for s in range(simulations):

    temp_memory = []
    round_wins = []
    print('*** learning phase %i / %i' % (s+1, simulations))
    for epoch in range(epochs):
        new_game(state)
        logger.info("*** start of a new game")
        print_state(state)

        for j in range(9):
            for i in range(4):
                result = play_once(state, lambda (possible_cards): \
                          choose(state,possible_cards,model,temp_memory,epsilon))

                assert i < 3 and result is None or \
                       i == 3 and result is not None

                if i == 3: # at the end of one round

                    round_wins.append(result)

                    if len(state['played']) == 36: 

                        # the game is finished, we can update the model
                        update_model_game_end(model,temp_memory,round_wins,epsilon)

    game_wins = pd.DataFrame({'game won':[]})
    round_wins = pd.DataFrame({'round won':[]})

    print('*** testing phase %i / %i' % (s+1, simulations))
    game_wins_idx = 0
    round_wins_idx = 0
    for epoch in range(test_runs):
        new_game(state)
        logger.info("*** start of a new game")
        print_state(state)

        for j in range(4*9):
            result = play_once(state, lambda (possible_cards): \
                      choose_for_test(state,possible_cards,model,temp_memory,epsilon))

            if result is not None:
                team = result['team']
                is_final = result['final']

                if is_final:
                    df = pd.DataFrame({'game won':[1 if team == 0 else 0]}, index=[game_wins_idx])
                    game_wins = game_wins.append(df)
                    game_wins_idx += 1
                else:
                    df = pd.DataFrame({'round won':[1 if team == 0 else 0]}, index=[round_wins_idx])
                    round_wins = round_wins.append(df)
                    round_wins_idx += 1


    games_won = 1.0 * len(game_wins[game_wins['game won'] == 1]) / test_runs
    # we have 8 rounds, excepted the final one that we don't count here
    rounds_won = 1.0 * len(round_wins[round_wins['round won'] == 1]) / (test_runs * 8)
    df = pd.DataFrame({'rounds won':[rounds_won], 'games won':[games_won]},index=[s])
    dp.display(df)
    stats = stats.append(df)
    
    if epsilon > 0.1:
        epsilon -= (1/simulations)

stats

*** learning phase 1 / 2
*** testing phase 1 / 2


,games won,rounds won
0,0.6,0.46125


*** learning phase 2 / 2
*** testing phase 2 / 2


,games won,rounds won
1,0.42,0.37625


,games won,rounds won
0,0.60,0.46125
1,0.42,0.37625


In [3]:
#model.predict(feat.reshape(1,110), batch_size=1)

# a = np.array([0,20,10,0.5])
# print("shape is %s" % a.shape)

# create_logger()
# create_logger()


# logger = get_logger()
# logger.warning('123 444')
#logger.warning('123 444')

# idx = 21
# idx / 9
# idx % 9

# mask = np.array([ 0.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,
#   1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.])
# qval = np.array([[  8.31833303e-01,   2.73417532e-01,   6.78057134e-01,   1.35389102e+00,
#    -9.99412751e+02,   7.50266671e-01,  -9.99387690e+02,   5.98161995e-01,
#    -9.99129153e+02,   1.29790694e-01,   6.95346236e-01,   4.18830216e-01,
#    -9.99768427e+02,  -4.85480996e-03,   2.25901946e-01,  -9.99599565e+02,
#     5.34388781e-01,   2.60302991e-01,  -9.99895701e+02,   7.67788053e-01,
#     1.85738713e-01,   1.25354931e-01,   4.80328113e-01,   2.87905961e-01,
#    -9.99341063e+02,   6.00367449e-02,   1.96240738e-01,   5.77371418e-01,
#    -9.99449866e+02,   2.62334257e-01,   5.46337962e-02,  -1.00027924e+03,
#     2.30767533e-01,   4.87363160e-01,   2.03288451e-01,   4.77576464e-01]])
# np.argmax(qval)

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_1 (Dense)                  (None, 220)           24420       dense_input_1[0][0]              
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 220)           0           dense_1[0][0]                    
____________________________________________________________________________________________________
dense_2 (Dense)                  (None, 300)           66300       activation_1[0][0]               
____________________________________________________________________________________________________
activation_2 (Activation)        (None, 300)           0           dense_2[0][0]                    
___________________________________________________________________________________________